<a href="https://colab.research.google.com/github/Uday-Trivedi/SoberCare/blob/main/PhaseSense_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q sentence-transformers chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currentl

In [4]:
import json
import numpy as np

# Embedding model
from sentence_transformers import SentenceTransformer

# Vector database
import chromadb

In [5]:
model = SentenceTransformer("BAAI/bge-small-en")

print("Model loaded successfully")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Model loaded successfully


In [7]:
# Load the vocabulary file that contains keywords and prototype sentences

with open("vocabulary.json", "r") as f:
    vocab = json.load(f)

print("Vocabulary loaded")

Vocabulary loaded


In [8]:
# Extract prototype sentences and keep track of their phases

documents = []
metadatas = []
ids = []

counter = 0

for phase in vocab:

    sentences = vocab[phase]["prototype_sentences"]

    for sentence in sentences:

        documents.append(sentence)
        metadatas.append({"phase": phase})
        ids.append(str(counter))

        counter += 1

print("Total prototype sentences:", len(documents))

Total prototype sentences: 100


In [9]:
# Convert all sentences into vector embeddings

embeddings = model.encode(documents)

print("Embeddings generated")
print("Vector shape:", embeddings.shape)

Embeddings generated
Vector shape: (100, 384)


In [10]:
# Create a Chroma client and collection

client = chromadb.Client()

collection = client.create_collection(
    name="phasesense_embeddings"
)

print("Chroma collection created")

Chroma collection created


In [11]:
# Store the prototype sentences and their embeddings

collection.add(
    documents=documents,
    embeddings=embeddings.tolist(),
    metadatas=metadatas,
    ids=ids
)

print("Embeddings stored in ChromaDB")

Embeddings stored in ChromaDB


In [12]:
# Check how many items are stored

count = collection.count()

print("Total vectors in database:", count)

Total vectors in database: 100


In [13]:
# Build a keyword dictionary for quick lookup

keyword_dict = {}

for phase in vocab:
    keyword_dict[phase] = set(vocab[phase]["keywords"])

print("Keyword dictionary ready")
print(len(keyword_dict))


Keyword dictionary ready
5


In [14]:
import re

def keyword_score(text):

    scores = {phase: 0 for phase in keyword_dict}

    # simple cleaning
    text = text.lower()
    words = re.findall(r'\b\w+\b', text)

    for phase in keyword_dict:
        for w in words:
            if w in keyword_dict[phase]:
                scores[phase] += 1

    return scores

In [15]:
def embedding_search(user_text, top_k=10):

    user_embedding = model.encode(user_text).tolist()

    results = collection.query(
        query_embeddings=[user_embedding],
        n_results=top_k
    )

    return results

In [16]:
def embedding_score(results):

    phase_scores = {phase: 0 for phase in keyword_dict}

    metadatas = results["metadatas"][0]
    distances = results["distances"][0]

    for meta, dist in zip(metadatas, distances):

        phase = meta["phase"]

        similarity = 1 - dist

        phase_scores[phase] += similarity

    return phase_scores

In [17]:
def predict_phase(user_text):

    # keyword signal
    k_scores = keyword_score(user_text)

    # embedding signal
    results = embedding_search(user_text)
    e_scores = embedding_score(results)

    final_scores = {}

    for phase in k_scores:
        final_scores[phase] = (0.7 * k_scores[phase]) + (0.5 * e_scores[phase])

    prediction = max(final_scores, key=final_scores.get)

    return prediction, final_scores

In [21]:
text = input()


phase, scores = predict_phase(text)

print("Input:", text)
print("Predicted Phase:", phase)
print("Scores:", scores)

i want to drink
Input: i want to drink
Predicted Phase: active_craving
Scores: {'denial_minimization': 0.0, 'cognitive_negotiation': 0.4059044122695923, 'active_craving': 4.572822088003159, 'heightened_vulnerability': 0.0, 'recovery_stabilization': 0.39633670449256897}
